In [45]:
import numpy as np
import soundfile as sf
from pathlib import Path

def mix_with_snr(clean_path, noise_path, output_path, target_snr=5):
    """
    混合语音与噪声至指定信噪比
    
    参数:
        clean_path (str): 纯净语音路径
        noise_path (str): 噪声路径
        output_path (str): 输出路径
        target_snr (float): 目标信噪比(dB)
    """
    # 读取音频文件
    clean, fs_clean = sf.read(clean_path)
    noise, fs_noise = sf.read(noise_path)

    # 统一采样率
    if fs_clean != fs_noise:
        raise ValueError("采样率不一致，请先进行重采样处理")
    
    # 统一声道数
    if clean.ndim != noise.ndim:
        noise = noise[:, 0] if noise.ndim > 1 else noise
        clean = clean[:, 0] if clean.ndim > 1 else clean

    # 获取音频长度（采样点数）
    total_samples = len(noise)
    clean_samples = len(clean)
    
    # 语音补零至与噪声等长
    padded_clean = np.zeros_like(noise)
    padded_clean[:clean_samples] = clean

    # 计算有效部分能量（前clean_samples个采样点）
    speech_energy = np.sum(padded_clean[:clean_samples]**2)
    noise_segment = noise[:clean_samples]
    noise_energy = np.sum(noise_segment**2)

    # 避免除零错误
    if speech_energy == 0:
        raise ValueError("语音信号能量为零")
    if noise_energy == 0:
        raise ValueError("噪声信号能量为零")

    # 计算信噪比调整系数
    snr_linear = 10**(target_snr / 10)
    scale_factor = np.sqrt(speech_energy / (snr_linear * noise_energy))

    # 调整全段噪声幅度
    scaled_noise = noise * scale_factor

    # 混合信号
    mixed = padded_clean + scaled_noise

    # 峰值归一化防止削波
    max_amp = np.max(np.abs(mixed))
    if max_amp > 1.0:
        mixed /= max_amp

    # 保存结果
    sf.write(output_path, mixed, fs_clean)

if __name__ == "__main__":
    # 输入输出路径
    base_dir = Path("TestData")
    clean_path = base_dir / "p263_156.wav"
    noise_path = base_dir / "AirConditioner_2.wav"
    output_path = base_dir / "mixed_5dB.wav"

    # 执行混合
    mix_with_snr(str(clean_path), 
                str(noise_path), 
                str(output_path),
                target_snr=0)

In [46]:
import numpy as np
import soundfile as sf
from pathlib import Path

def mix_with_snr_Time(clean_path, noise_path, output_path, target_snr=5,Time = 2):
    """
    混合语音与噪声至指定信噪比
    
    参数:
        clean_path (str): 纯净语音路径
        noise_path (str): 噪声路径
        output_path (str): 输出路径
        target_snr (float): 目标信噪比(dB)
    """
    # 读取音频文件
    clean, fs_clean = sf.read(clean_path)
    noise, fs_noise = sf.read(noise_path)

    # 统一采样率
    if fs_clean != fs_noise:
        raise ValueError("采样率不一致，请先进行重采样处理")
    
    # 统一声道数
    if clean.ndim != noise.ndim:
        noise = noise[:, 0] if noise.ndim > 1 else noise
        clean = clean[:, 0] if clean.ndim > 1 else clean

    # 获取音频长度（采样点数）
    total_samples = len(noise)
    clean_samples = len(clean)
    All_Samples = fs_noise * Time
    if(clean_samples >= All_Samples):
        padded_clean = clean[:All_Samples]
    else:
        # 语音补零至与噪声等长
        padded_clean = np.zeros(All_Samples)
        padded_clean[:clean_samples] = clean
    if(total_samples >= All_Samples):
        padded_noise = noise[:All_Samples]
    else:
        # 语音补零至与噪声等长
        padded_noise = np.zeros(All_Samples)
        padded_noise[:total_samples] = noise

    # 计算有效部分能量（前clean_samples个采样点）
    speech_energy = np.sum(padded_clean**2)
    noise_segment = noise[:clean_samples]
    noise_energy = np.sum(noise_segment**2)

    # 避免除零错误
    if speech_energy == 0:
        raise ValueError("语音信号能量为零")
    if noise_energy == 0:
        raise ValueError("噪声信号能量为零")

    # 计算信噪比调整系数
    snr_linear = 10**(target_snr / 10)
    scale_factor = np.sqrt(speech_energy / (snr_linear * noise_energy))

    # 调整全段噪声幅度
    scaled_noise = padded_noise * scale_factor

    # 混合信号
    mixed = padded_clean + scaled_noise

    # 峰值归一化防止削波
    max_amp = np.max(np.abs(mixed))
    if max_amp > 1.0:
        mixed /= max_amp

    # 保存结果
    sf.write(output_path, mixed, fs_clean)
if __name__ == "__main__":
    # 输入输出路径
    base_dir = Path("TestData")
    clean_path = base_dir / "p263_156.wav"
    noise_path = base_dir / "AirConditioner_3.wav"
    output_path = base_dir / "mixed_0dB.wav"

    # 执行混合
    mix_with_snr_Time(str(clean_path), 
                str(noise_path), 
                str(output_path),
                target_snr=5,
                Time=20)